In [39]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
from tweepy import API
from tweepy import Cursor
import twitter_credentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
import re

In [31]:
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.API, twitter_credentials.API_SECRET_KEY)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

In [32]:
class TwittertListener(StreamListener):
    """
    This listener appends data to the passed filename
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename
        
    def on_data(self, tweet):
        try:
            #print(tweet)
            with open (self.fetched_tweets_filename, 'a') as tf:
                tf.write(tweet)
            
        except Exception as e:
            print(e)
        
        finally:
            return True
            

    def on_error(self, status):
        if status == 420:
                # Returning False on_data method in case rate limit occurs
                return False
        print(status)


In [33]:
class TwitterStreamer():
    """
    Class for streaming and processing live tweets
    """
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)
        stream.filter(track = hash_tag_list)        

In [7]:
class TwitterClient():
    def __init__(self, twitter_user = None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user
        
    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id = self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id = self.twitter_user).items(num_tweets):
            friend_list.append(friend)
        return friend_list
    
    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id = self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return(home_timeline_tweets)
    
    def get_twitter_client_api(self):
        return(self.twitter_client)

In [41]:
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing tweets
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data = [tweet.text for tweet in tweets], columns = ['tweets'])
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([(tweet.source) for tweet in tweets])
        df['likes'] = np.array([(tweet.favorite_count) for tweet in tweets])
        df['retweets'] = np.array([(tweet.retweet_count) for tweet in tweets])
        return df
    
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

In [44]:
if __name__ == '__main__':
    twitter_client = TwitterClient()
    api = twitter_client.get_twitter_client_api()
    tweets = api.user_timeline(screen_name="realDonaldTrump", count = 200)
    tweet_analyzer = TweetAnalyzer()
    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = df['tweets'].apply(tweet_analyzer.analyze_sentiment)
    print(df.head())

                                              tweets                   id  \
0  Good news on Covid Relief Bill. Information to...  1343336121040777217   
1  See you in Washington, DC, on January 6th. Don...  1343328708963299338   
2  On behalf of two GREAT Senators, @sendavidperd...  1343328283824447488   
3                A must see! https://t.co/WbLDGpaHJX  1343069190433943564   
4       RT @realDonaldTrump: https://t.co/L3tOUDiT0G  1343069079867957248   

   len                date              source   likes  retweets  sentiment  
0   54 2020-12-27 23:21:03  Twitter for iPhone  225374     38898          1  
1   80 2020-12-27 22:51:36  Twitter for iPhone  301527     66267          0  
2  143 2020-12-27 22:49:55  Twitter for iPhone  179641     37705          1  
3   35 2020-12-27 05:40:22  Twitter for iPhone  187427     57985          0  
4   44 2020-12-27 05:39:56  Twitter for iPhone       0     84462          0  
